In [79]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import Select
import re
import time
from selenium.webdriver.chrome.options import Options


# Start and login


In [15]:
# open the browser and visit the url
browser = webdriver.Chrome()
browser.get('http://www.khoa.go.kr/oceangrid/gis/category/reference/distribution.do#none')

In [16]:
# click 로그인
login_button =  browser.find_element(By.XPATH,"//a[text()='로그인']")
login_button.click()

In [17]:
# in the login page, input username and password and login
id = ""
pw = ""
id_box = browser.find_element(By.ID,"user_id")
pw_box = browser.find_element(By.ID,"user_passwd") 
#input the id into the id cell
id_box.send_keys(id)
pw_box.send_keys(pw)
login_button_at_login_page = browser.find_element(By.ID,"btn_login")
login_button_at_login_page.click()

# basic info input

In [20]:
# select the time interval
start_year_button = browser.find_element(By.ID,"searchPreYear1")
start_month_button = browser.find_element(By.ID,"searchPreMonth1")
end_year_button = browser.find_element(By.ID,"searchPreYear2")
end_month_button = browser.find_element(By.ID,"searchPreMonth2")

In [21]:
# the timeframe gap between start and end should be less than 2 years
start_year, start_month, end_year, end_month = "2000", "01", "2001", "12"


In [23]:
Select(start_year_button).select_by_value(start_year)
Select(start_month_button).select_by_value(start_month)
Select(end_year_button).select_by_value(end_year)
Select(end_month_button).select_by_value(end_month)


In [27]:
# click to select the item you wanna download, in our current task, it is the "수온" item
item_button = browser.find_element(By.ID,"chkCodeTemp")
item_button.click()

In [28]:
# input the location keyword. In our current task, it is "모슬포"
keyword_box = browser.find_element(By.ID,"searchKey")
keyword_box.send_keys("모슬포")

In [29]:
# click the 검색 button to show the result
search_button = browser.find_element(By.XPATH,'//*[@id="content"]/div/div[3]/div[1]/p/a/img')
search_button.click()

# Download

In [32]:
# click the 번호 button to select all items in the current page
beonho_button = browser.find_element(By.ID,'checkboxAll')
beonho_button.click()

In [33]:
# click the 다운로드 button to download the data
download_button = browser.find_element(By.NAME,'zipdown')
download_button.click()

In [37]:
# get the page's html
html = browser.page_source
pattern = r'a href="javascript:goPage(.*?);">.*?</a>'
matches = re.findall(pattern, html)
len(matches)
# len(matches) - 3 = amount of pages in current search result
# len(matches) - 4 = how many times we should click the next page button


6

In [40]:
# click the next page button
next_page_button = browser.find_element(By.CSS_SELECTOR,'img[alt="다음페이지"]')
next_page_button.click()

# Now we have components, let's combine them

There are 2 iterations for download all data: 1) In the time interval selection, and 2) In the downloadpage

In [88]:

def download_data(start_year_int = 2010, start_month = '01', end_year_int = 2011, end_month = '12', location_keyword = "모슬포", final_year = 2024):
    # Set Chrome options to keep the browser window open
    chrome_options = Options()
    chrome_options.add_experimental_option("detach", True)
    browser = webdriver.Chrome(options=chrome_options)
    browser.get('http://www.khoa.go.kr/oceangrid/gis/category/reference/distribution.do#none')

    # click 로그인
    login_button =  browser.find_element(By.XPATH,"//a[text()='로그인']")
    login_button.click()
    time.sleep(2)
    id = ""
    time.sleep(2)

    pw = ""
    id_box = browser.find_element(By.ID,"user_id")
    pw_box = browser.find_element(By.ID,"user_passwd")  
    #input the id into the id cell
    id_box.send_keys(id)
    time.sleep(2)
    pw_box.send_keys(pw)
    time.sleep(2)
    login_button_at_login_page = browser.find_element(By.ID,"btn_login")
    login_button_at_login_page.click()
    time.sleep(2)

    # input the location keyword. In our current task, it is "모슬포"
    keyword_box = browser.find_element(By.ID,"searchKey")
    keyword_box.send_keys(location_keyword)

    
    #click to select the item you wanna download, in our current task, it is the "수온" item
    item_button = browser.find_element(By.ID,"chkCodeTemp")
    item_button.click()
    # select the time interval
    
    while start_year_int < final_year + 1 :
        start_year_button = browser.find_element(By.ID,"searchPreYear1")
        start_month_button = browser.find_element(By.ID,"searchPreMonth1")
        end_year_button = browser.find_element(By.ID,"searchPreYear2")
        end_month_button = browser.find_element(By.ID,"searchPreMonth2")
        # the timeframe gap between start and end should be less than 2 years
        start_year = str(start_year_int)
        end_year = str(end_year_int)
        Select(start_year_button).select_by_value(start_year)
        Select(start_month_button).select_by_value(start_month)
        Select(end_year_button).select_by_value(end_year)
        Select(end_month_button).select_by_value(end_month)


        # click the 검색 button to show the result
        search_button = browser.find_element(By.XPATH,'//*[@id="content"]/div/div[3]/div[1]/p/a/img')
        search_button.click()

        start_year_int, end_year_int = start_year_int + 2, end_year_int + 2

        time.sleep(2)
        try:
            popup_button = browser.find_element(By.CLASS_NAME,'ui-dialog-buttonset')
            popup_button.click()
        except:
            pass


        # get the page's html
        html = browser.page_source
        pattern = r'a href="javascript:goPage(.*?);">.*?</a>'
        matches = re.findall(pattern, html)
        
        # len(matches) - 3 = amount of pages in current search result
        # len(matches) - 4 = how many times we should click the next page button
        next_page_button_click_times = len(matches) - 4

        # click the 번호 button to select all items in the current page
        beonho_button = browser.find_element(By.ID,'checkboxAll')
        beonho_button.click()

        # click the 다운로드 button to download the data
        download_button = browser.find_element(By.NAME,'zipdown')
        download_button.click()
        time.sleep(2)
        
        for i in range(next_page_button_click_times):
            # click the next page button
            next_page_button = browser.find_element(By.CSS_SELECTOR,'img[alt="다음페이지"]')
            next_page_button.click()
            time.sleep(5)

            beonho_button = browser.find_element(By.ID,'checkboxAll')
            beonho_button.click()

            # click the 다운로드 button to download the data
            download_button = browser.find_element(By.NAME,'zipdown')
            download_button.click()
            time.sleep(2)
            try:
                popup_button = browser.find_element(By.CLASS_NAME,'ui-dialog-buttonset')
                popup_button.click()
            except:
                pass
        time.sleep(2)

In [89]:
download_data()

KeyboardInterrupt: 

In [7]:
import imageio

input_file = r'C:\Lu\Lu4Paper\Crawler\MOF\run_example.gif'
output_file = r'C:\Lu\Lu4Paper\Crawler\MOF\run_example1.gif'

with imageio.get_reader(input_file) as reader:
    # Get the frame rate from the metadata, or set a default value if 'fps' key is not present
    try:
        fps = reader.get_meta_data()['fps']
    except KeyError:
        fps = 5  # Default frame rate if 'fps' key is not present

    writer = imageio.get_writer(output_file, fps=fps)

    for frame in reader:
        writer.append_data(frame)

    writer.close()
